In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
%config InlineBackend.figure_formats = ['retina'] 
import statsmodels.api as sm

import seaborn as sns

In [99]:
quality = pd.read_csv('../oecd_quality_uspto_jan20.csv')

In [68]:
info_companies = pd.read_csv("./Data/info_companies.csv")

In [15]:
info_patents = pd.read_pickle('../info_patents')
info_patents = info_patents.dropna()

sold_patents = pd.read_csv('data/documentid_red.csv')

In [16]:
mlb = MultiLabelBinarizer()
dummies = pd.DataFrame(mlb.fit_transform(info_patents.ipc_section),columns=mlb.classes_, index=info_patents.index)
dummies = dummies[['A','B','C','D','E','F','G','H']]

for el in list(dummies.columns):
    info_patents[el]=dummies[el]

patents_to_drop = info_patents[(info_patents.A==0) & (info_patents.B==0) & (info_patents.C==0) & (info_patents.D==0) & (info_patents.E==0) & (info_patents.F==0) & (info_patents.G==0) & (info_patents.H==0)].patent_number
info_patents = info_patents[~info_patents.patent_number.isin(list(patents_to_drop))]

In [21]:
info_patents = info_patents.reset_index()
multi_ipc = [0] * len(info_patents)
for i in list(info_patents.loc[info_patents.ipc_section.apply(lambda x: len(x)>1)].index):
    multi_ipc[i]=1
info_patents["multi_ipc"] = multi_ipc

In [25]:
info_patents.patent_number = pd.to_numeric(info_patents.patent_number,errors='coerce')

In [26]:
info_patents

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc
0,0,5387603,1995-02-07,433,"[A, C]",[5330279e-a21e-438c-80d1-7e6ae793c092],"[16, 14]",1.0,1,0,1,0,0,0,0,0,1
1,1,5771295,1998-06-23,552,[H],[2986e46e-7937-477f-9bc5-1f5eec05183f],[2],1.0,0,0,0,0,0,0,0,1,0
2,2,5585587,1996-12-17,832,[G],[f7152515-d420-42d9-a7b6-3a2796bc5d10],"[34, 2, 10]",1.0,0,0,0,0,0,0,1,0,0
3,3,5321490,1994-06-14,568,[G],[ec72750e-f789-4749-91f7-6292f7c6fda7],"[10, 12, 29]",1.0,0,0,0,0,0,0,1,0,0
4,4,5750497,1998-05-12,1161,"[A, C]",[9a3afa40-66e5-4921-b831-64287f2052e0],"[16, 15]",1.0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823827,1915705,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0
1823828,1915707,4192319,1980-03-11,893,[A],[fa543ac8-5ee8-4999-b949-6c8a3dbae65d],[13],0.0,1,0,0,0,0,0,0,0,0
1823829,1915708,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0
1823830,1915709,4191891,1980-03-04,481,[H],[f2d3dc32-b29c-4c55-ab6d-736b4ee8b96b],[13],0.0,0,0,0,0,0,0,0,1,0


In [28]:
info_patents = info_patents.dropna()
info_patents = info_patents[info_patents.patent_date >= '2000-01-01']
info_patents = info_patents[info_patents.patent_date <= '2019-01-01']
info_patents = info_patents.reset_index(drop=True)

In [29]:
info_patents = info_patents[info_patents.patent_processing_time.apply(lambda x: str(x).isdigit())].reset_index(drop=True)
info_patents.patent_processing_time = info_patents.patent_processing_time.astype(int)

In [30]:
info_patents

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc
0,447,6127678,2000-10-03,329,[H],[019b9aba-95cd-43bf-a8a5-e700229fddac],"[28, 25]",1.0,0,0,0,0,0,0,0,1,0
1,449,6073289,2000-06-13,908,[A],[e5fc32c3-a913-455a-94ce-9932d30eefff],[13],1.0,1,0,0,0,0,0,0,0,0
2,450,6073290,2000-06-13,944,[A],[3860ed0c-8e84-4bed-ba0b-9d7a57d3a6b9],"[29, 13]",1.0,1,0,0,0,0,0,0,0,0
3,451,6064927,2000-05-16,995,"[B, G]",[996d1179-2d0d-4a05-ac9f-32c3eb34fb69],"[10, 32]",1.0,0,1,0,0,0,0,1,0,1
4,453,6029495,2000-02-29,651,[G],[0f63d31a-dac4-4326-afe8-e0d9ca77e826],[10],1.0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903370,1915702,8386963,2013-02-26,1370,[G],[4674eaf4-2927-46ab-ae18-9d92bfcce864],[6],0.0,0,0,0,0,0,0,1,0,0
903371,1915703,8386259,2013-02-26,2252,[G],[1b7df99f-bb3a-4026-a4ab-f5f89558b405],"[6, 3, 4]",0.0,0,0,0,0,0,0,1,0,0
903372,1915705,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0
903373,1915708,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0


In [31]:
test = info_patents.append([info_patents]*18)

In [32]:
test

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc
0,447,6127678,2000-10-03,329,[H],[019b9aba-95cd-43bf-a8a5-e700229fddac],"[28, 25]",1.0,0,0,0,0,0,0,0,1,0
1,449,6073289,2000-06-13,908,[A],[e5fc32c3-a913-455a-94ce-9932d30eefff],[13],1.0,1,0,0,0,0,0,0,0,0
2,450,6073290,2000-06-13,944,[A],[3860ed0c-8e84-4bed-ba0b-9d7a57d3a6b9],"[29, 13]",1.0,1,0,0,0,0,0,0,0,0
3,451,6064927,2000-05-16,995,"[B, G]",[996d1179-2d0d-4a05-ac9f-32c3eb34fb69],"[10, 32]",1.0,0,1,0,0,0,0,1,0,1
4,453,6029495,2000-02-29,651,[G],[0f63d31a-dac4-4326-afe8-e0d9ca77e826],[10],1.0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903370,1915702,8386963,2013-02-26,1370,[G],[4674eaf4-2927-46ab-ae18-9d92bfcce864],[6],0.0,0,0,0,0,0,0,1,0,0
903371,1915703,8386259,2013-02-26,2252,[G],[1b7df99f-bb3a-4026-a4ab-f5f89558b405],"[6, 3, 4]",0.0,0,0,0,0,0,0,1,0,0
903372,1915705,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0
903373,1915708,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0


In [33]:
test = test.sort_values(by='patent_number',ascending='True')

In [34]:
test

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc
421801,945516,6009558,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0
421801,945516,6009558,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0
421801,945516,6009558,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0
421801,945516,6009558,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0
421801,945516,6009558,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421678,929073,10172260,2019-01-01,378,[H],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[2],0.0,0,0,0,0,0,0,0,1,0
421678,929073,10172260,2019-01-01,378,[H],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[2],0.0,0,0,0,0,0,0,0,1,0
421678,929073,10172260,2019-01-01,378,[H],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[2],0.0,0,0,0,0,0,0,0,1,0
421678,929073,10172260,2019-01-01,378,[H],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[2],0.0,0,0,0,0,0,0,0,1,0


In [35]:
test["Year"] = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]*len(info_patents)

In [36]:
test = test.set_index(['patent_number','Year'])
test.reset_index(level=['Year'], inplace=True)

In [37]:
test['granted']=pd.DatetimeIndex(test.patent_date).year

In [39]:
#test.granted=test.granted.apply(lambda x: x==Year)

In [40]:
test.granted = test.granted==test.Year

In [41]:
test.head(20)

,Year,index,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc,granted
patent_number,,,,,,,,,,,,,,,,,,
6009558,2000,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,True
6009558,2001,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2002,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2003,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2004,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2005,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2006,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2007,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False
6009558,2008,945516,2000-01-04,1001,[A],[2cbf39b7-3c7c-4a37-ba74-2b5d9e26b013],"[13, 34]",0.0,1,0,0,0,0,0,0,0,0,False


In [100]:
quality
#np.unique([len(info_patents.patent_number[i]) for i in range(0,903446)],return_counts=True)

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,US03951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,US03956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,US03960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,US03963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,US03963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8574380,516842427,US2019230828,2019,1,1,2,1,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574381,516842321,US2019230829,2016,2,0,1,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574382,516842367,US2019230830,2016,2,0,1,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574383,516851404,US2019230831,2016,25,1,2,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
quality = quality[quality.pub_nbr.apply(lambda x: len(x)==10)]
quality

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,US03951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,US03956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,US03960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,US03963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,US03963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270183,477846335,US10362716,2015,2,1,2,1,1369.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,4.0,0.094294,0.075252
6270184,498406497,US10362717,2016,21,1,2,4,1070.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,3.0,0.273504,0.191243
6270185,496339505,US10362718,2017,1,1,3,3,592.0,0,0,...,NaN,1,1,0,0.5,NaN,NaN,2.0,0.224706,0.203709
6270186,479265472,US10362719,2014,2,0,1,4,1938.0,6,0,...,NaN,0,0,0,NaN,0.617284,0.222222,5.0,0.128205,0.086840


In [103]:
quality.pub_nbr = quality.pub_nbr.astype(str)
quality.pub_nbr = quality.pub_nbr.str.strip('US')
quality

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,03951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,03956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,03960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,03963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,03963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270183,477846335,10362716,2015,2,1,2,1,1369.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,4.0,0.094294,0.075252
6270184,498406497,10362717,2016,21,1,2,4,1070.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,3.0,0.273504,0.191243
6270185,496339505,10362718,2017,1,1,3,3,592.0,0,0,...,NaN,1,1,0,0.5,NaN,NaN,2.0,0.224706,0.203709
6270186,479265472,10362719,2014,2,0,1,4,1938.0,6,0,...,NaN,0,0,0,NaN,0.617284,0.222222,5.0,0.128205,0.086840


In [105]:
quality.pub_nbr = quality.pub_nbr.astype(int)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [65]:
info_patents

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,D,E,F,G,H,multi_ipc
0,447,6127678,2000-10-03,329,[H],[019b9aba-95cd-43bf-a8a5-e700229fddac],"[28, 25]",1.0,0,0,0,0,0,0,0,1,0
1,449,6073289,2000-06-13,908,[A],[e5fc32c3-a913-455a-94ce-9932d30eefff],[13],1.0,1,0,0,0,0,0,0,0,0
2,450,6073290,2000-06-13,944,[A],[3860ed0c-8e84-4bed-ba0b-9d7a57d3a6b9],"[29, 13]",1.0,1,0,0,0,0,0,0,0,0
3,451,6064927,2000-05-16,995,"[B, G]",[996d1179-2d0d-4a05-ac9f-32c3eb34fb69],"[10, 32]",1.0,0,1,0,0,0,0,1,0,1
4,453,6029495,2000-02-29,651,[G],[0f63d31a-dac4-4326-afe8-e0d9ca77e826],[10],1.0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903370,1915702,8386963,2013-02-26,1370,[G],[4674eaf4-2927-46ab-ae18-9d92bfcce864],[6],0.0,0,0,0,0,0,0,1,0,0
903371,1915703,8386259,2013-02-26,2252,[G],[1b7df99f-bb3a-4026-a4ab-f5f89558b405],"[6, 3, 4]",0.0,0,0,0,0,0,0,1,0,0
903372,1915705,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0
903373,1915708,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0,0,0,0,0,0,0,1,0,0


In [66]:
assignee_id1 = list(info_patents.assignee_id)
for index, i in enumerate(assignee_id1):
    assignee_id1[index] = i[0]
    
info_patents["assignee_id1"] = assignee_id1

In [69]:
test = info_patents.set_index('assignee_id1').join(info_companies.set_index('assignee_id'), rsuffix='_other')

sold_patents['sold'] = [1] * len(sold_patents)
sold_patents = sold_patents[['sold','grant_doc_num']]

test3 = test.set_index('patent_number').join(sold_patents.set_index('grant_doc_num'), rsuffix='_other')

In [70]:
test

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,...,nb_marked_patents,proportion_marked_patents,A_other,B_other,C_other,D_other,E_other,F_other,G_other,H_other
0004b544-00ba-4d48-a9b1-072bfcac8859,2165,6494858,2002-12-17,902,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,3,0.500,6,0,0,0,0,0,0,0
0004b544-00ba-4d48-a9b1-072bfcac8859,4291,6994685,2006-02-07,1876,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,3,0.500,6,0,0,0,0,0,0,0
0004b544-00ba-4d48-a9b1-072bfcac8859,6881,7549973,2009-06-23,2626,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,3,0.500,6,0,0,0,0,0,0,0
00141147-703c-4b54-9eee-f2c8bb9f2f93,477,6086660,2000-07-11,526,[C],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[19],1.0,0,0,...,28,0.001,515,9695,1121,147,22,585,12334,8430
00141147-703c-4b54-9eee-f2c8bb9f2f93,937,6019470,2000-02-01,973,[G],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[9],1.0,0,0,...,28,0.001,515,9695,1121,147,22,585,12334,8430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0,1041065,6184718,2001-02-06,580,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,14,0.326,1,22,0,0,1,2,18,10
ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0,1096828,6288593,2001-09-11,616,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,14,0.326,1,22,0,0,1,2,18,10
ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0,1196493,6469541,2002-10-22,321,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,14,0.326,1,22,0,0,1,2,18,10
ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0,1913897,8382401,2013-02-26,1238,[B],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[25],0.0,0,1,...,14,0.326,1,22,0,0,1,2,18,10


In [83]:
test['sold']=[0]*len(test)
test = test.reset_index(drop=True)
list_test = list(sold_patents.grant_doc_num)
test.loc[test[test.patent_number.isin(list_test)].index,'sold']=1

In [84]:
test.sold.value_counts()

0    896037
1      7338
Name: sold, dtype: int64

In [86]:
test

,index,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,...,proportion_marked_patents,A_other,B_other,C_other,D_other,E_other,F_other,G_other,H_other,sold
0,2165,6494858,2002-12-17,902,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,0.500,6,0,0,0,0,0,0,0,0
1,4291,6994685,2006-02-07,1876,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,0.500,6,0,0,0,0,0,0,0,0
2,6881,7549973,2009-06-23,2626,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,...,0.500,6,0,0,0,0,0,0,0,0
3,477,6086660,2000-07-11,526,[C],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[19],1.0,0,0,...,0.001,515,9695,1121,147,22,585,12334,8430,0
4,937,6019470,2000-02-01,973,[G],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[9],1.0,0,0,...,0.001,515,9695,1121,147,22,585,12334,8430,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903370,1041065,6184718,2001-02-06,580,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,0.326,1,22,0,0,1,2,18,10,0
903371,1096828,6288593,2001-09-11,616,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,0.326,1,22,0,0,1,2,18,10,0
903372,1196493,6469541,2002-10-22,321,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,...,0.326,1,22,0,0,1,2,18,10,0
903373,1913897,8382401,2013-02-26,1238,[B],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[25],0.0,0,1,...,0.326,1,22,0,0,1,2,18,10,0


In [106]:
quality

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,3951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,3956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,3960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,3963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,3963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270183,477846335,10362716,2015,2,1,2,1,1369.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,4.0,0.094294,0.075252
6270184,498406497,10362717,2016,21,1,2,4,1070.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,3.0,0.273504,0.191243
6270185,496339505,10362718,2017,1,1,3,3,592.0,0,0,...,NaN,1,1,0,0.5,NaN,NaN,2.0,0.224706,0.203709
6270186,479265472,10362719,2014,2,0,1,4,1938.0,6,0,...,NaN,0,0,0,NaN,0.617284,0.222222,5.0,0.128205,0.086840


In [107]:
final_df = test.set_index('patent_number').join(quality.set_index('pub_nbr'), rsuffix='_other')




In [108]:
final_df

,index,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,A,B,C,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
patent_number,,,,,,,,,,,,,,,,,,,,,
6494858,2165,2002-12-17,902,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,0,...,NaN,12.0,14.0,0.0,0.806836,0.908163,0.333333,14.0,0.329110,0.359168
6994685,4291,2006-02-07,1876,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,0,...,NaN,1.0,2.0,0.0,0.666667,0.943755,0.432432,15.0,0.332287,0.344432
7549973,6881,2009-06-23,2626,[A],[0004b544-00ba-4d48-a9b1-072bfcac8859],[13],1.0,1,0,0,...,NaN,3.0,5.0,0.0,0.722222,0.909600,0.320000,12.0,0.309382,0.316171
6086660,477,2000-07-11,526,[C],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[19],1.0,0,0,1,...,NaN,3.0,5.0,0.0,0.273646,0.628693,0.490566,12.0,0.175732,0.265933
6019470,937,2000-02-01,973,[G],[00141147-703c-4b54-9eee-f2c8bb9f2f93],[9],1.0,0,0,0,...,NaN,11.0,21.0,0.0,0.218549,0.578125,0.312500,14.0,0.255127,0.276344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6184718,1041065,2001-02-06,580,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,0,...,NaN,14.0,14.0,0.0,0.689980,0.636487,0.361702,6.0,0.325401,0.383886
6288593,1096828,2001-09-11,616,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,0,...,NaN,11.0,11.0,0.0,0.789371,0.838374,0.456522,5.0,0.302771,0.371471
6469541,1196493,2002-10-22,321,[H],[ffeb6305-fbc7-40d9-9cc4-6e8f010fd6a0],[5],0.0,0,0,0,...,NaN,4.0,5.0,0.0,0.625000,0.364198,0.777778,5.0,0.218349,0.322925
